In [ ]:
!pip install d2l==1.0.0-beta0

Bạn có thể nhận thấy rằng việc triển khai từ đầu và triển khai ngắn gọn bằng cách sử dụng chức năng khung khá giống nhau trong trường hợp hồi quy. Điều này cũng đúng với việc phân loại. Vì nhiều mô hình trong cuốn sách này đề cập đến việc phân loại, nên cần thêm các chức năng để hỗ trợ cài đặt này một cách cụ thể. Phần này cung cấp một lớp cơ sở cho các mô hình phân loại để đơn giản hóa mã trong tương lai.

In [ ]:
import torch
from d2l import torch as d2l

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# 4.3.1. Lớp phân loại

Ta định nghĩa lớp `Classifier` ở dưới. Với phương thức `validation_step` ta báo cáo cả loss value và độ chính xác phân loại ở validation batch. Ta sẽ vẽ một bản cập nhật cho mỗi num_val_batches. Lợi ích của nó là tạo ra tổn thất trung bình và độ chính xác trên toàn bộ dữ liệu. Những con số trung bình này không chính xác nếu batch cuối cùng chứa ít ví dụ hơn, nhưng ta sẽ skip qua sự khác biệt nhỏ này. 

In [ ]:
class Classifier(d2l.Module):
  def validation_step(self, batch):
    Y_hat = self(*batch[:-1])
    self.plot('loss', self.loss(Y_hat, batch[-1]), train = False)
    self.plot('acc', self.accuracy(Y_hat, batch[-1]), train = False)
    

Theo mặc định, chúng tôi sử dụng trình tối ưu hóa giảm dần độ dốc ngẫu nhiên, hoạt động trên các gói nhỏ, giống như chúng tôi đã làm trong bối cảnh hồi quy tuyến tính.

In [ ]:
@d2l.add_to_class(d2l.Module)
def configure_optimizers(self):
  return torch.optim.SGD(self.parameters(), lr = self.lr)

# 4.3.2. Sự chính xác

Với phân phối xác suất dự đoán y_hat, chúng tôi thường chọn lớp có xác suất dự đoán cao nhất bất cứ khi nào chúng tôi phải đưa ra một dự đoán khó. Thật vậy, nhiều ứng dụng yêu cầu chúng ta phải lựa chọn. Chẳng hạn, Gmail phải phân loại email thành “Chính”, “Xã hội”, “Cập nhật”, “Diễn đàn” hoặc “Thư rác”. Nó có thể ước tính xác suất nội bộ, nhưng vào cuối ngày, nó phải chọn một trong số các lớp.

Khi các dự đoán nhất quán với lớp nhãn y, chúng là chính xác. Độ chính xác phân loại là tỷ lệ của tất cả các dự đoán đúng. Mặc dù có thể khó tối ưu hóa độ chính xác một cách trực tiếp (không thể vi phân), nhưng đây thường là thước đo hiệu suất mà chúng tôi quan tâm nhất. Nó thường là số lượng có liên quan trong điểm chuẩn. Như vậy, chúng tôi gần như sẽ luôn báo cáo nó khi đào tạo bộ phân loại.

Độ chính xác được tính như sau. Đầu tiên, nếu y_hatlà một ma trận, chúng tôi giả sử rằng chiều thứ hai lưu trữ điểm dự đoán cho mỗi lớp. Chúng tôi sử dụng argmaxđể có được lớp dự đoán theo chỉ mục cho mục nhập lớn nhất trong mỗi hàng. Sau đó, chúng tôi so sánh lớp dự đoán với sự thật cơ bản ytheo từng yếu tố. Vì toán tử đẳng thức == nhạy cảm với các loại dữ liệu, nên chúng tôi chuyển đổi y_hatloại dữ liệu của ' để khớp với loại dữ liệu của y. Kết quả là một tensor chứa các giá trị 0 (false) và 1 (true). Lấy tổng mang lại số dự đoán đúng.

In [ ]:
@d2l.add_to_class(Classifier)
def accuracy(self, Y_hat, Y, averaged = True):
  Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
  preds = Y_hat.argmax(axis=1).type(Y.dtype)
  compare = (preds == Y.reshape(-1)).type(torch.float32)
  return compare.mean() if averaged else compare